# creating dataset

In [1]:
!pip install torchviz

In [2]:
from torch_geometric.data import Dataset, Data
from tqdm import tqdm
from torchviz import make_dot
import torch
import os
import vtk
import numpy as np


# custom logger

In [3]:
import datetime
import pandas as pd

class MLLogger:
    def __init__(self, model_name):
        self.model_name = model_name
        self.log = pd.DataFrame(columns=['timestamp', 'epoch', 'train_loss', 'train_accuracy', 'test_loss', 'test_accuracy'])

    def log_metrics(self, epoch, train_loss, train_accuracy, test_loss, test_accuracy):
        now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        row = { 'timestamp': now,
                                'epoch': epoch, 
                                'train_loss': train_loss, 
                                'train_accuracy': train_accuracy, 
                                'test_loss': test_loss, 
                                'test_accuracy': test_accuracy}
        # self.log = self.log.append(row, ignore_index=True)
        # use concat instead of append to avoid copying the whole dataframe
        self.log = pd.concat([self.log, pd.DataFrame([row])], ignore_index=True)

    def save_log(self, file_path):
        logs_folder = "../.morsegram/logs"
        if not os.path.exists(logs_folder):
            os.makedirs(logs_folder)
        self.log.to_csv(os.path.join(logs_folder, self.model_name + "_" + file_path), index=False)


In [4]:
class Triples():
    def __init__(self):
        self.cp_id1 = 0
        self.cp_id2 = 0
        self.cp_id3 = 0
        self.pos1 = (0, 0, 0)
        self.pos2 = (0, 0, 0)
        self.pos3 = (0, 0, 0)
        self.val1 = 0
        self.val2 = 0
        self.val3 = 0
        self.alpha = 0


# Dataset for Morsegram Graphs

In [5]:
class MorseGramDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        self.files_metadata_dict = {}
        super(MorseGramDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        list_data_folders = []
        # adding folder names
        for file in os.listdir(self.root + "/raw/"):
            if os.path.isdir(self.root + "/raw/" + file):
                list_data_folders.append(file)
        output_files = []
        for folder in list_data_folders:
            for label in ['0', '1']:
                for file in os.listdir(self.root + "/raw/" + folder + "/simplified/" + label + "/"):
                    if file.endswith(".vtp"):
                        output_files.append(f'data_{len(output_files)}.pt')

        return output_files

    def download(self):
        pass

    def process(self):
        list_data_folders = []
        start_file_no = 0
        # adding folder names
        for file in os.listdir(self.root + "/raw/"):
            if os.path.isdir(self.root + "/raw/" + file):
                list_data_folders.append(file)

        for folder in tqdm(list_data_folders):
            start_file_no += self._process_single_data_folder(
                folder, start_file_no)

        # save metadata file
        with open(self.processed_dir + "/metadata.txt", "w") as f:
            f.write(str(self.files_metadata_dict))

    def _process_single_data_folder(self, folder_name, start_file_no):

        reader = vtk.vtkXMLPolyDataReader()
        reader.SetFileName(self.root + "/raw/" + folder_name + "/cps_3.vtp")
        reader.Update()
        self.cp3_data = reader.GetOutput()

        file_count = 0
        for label in ['0', '1']:
            for file in os.listdir(self.root + "/raw/" + folder_name + "/simplified/" + label + "/"):
                reader = vtk.vtkXMLPolyDataReader()
                reader.SetFileName(
                    self.root + "/raw/" + folder_name + "/simplified/" + label + "/" + file)
                reader.Update()
                data = reader.GetOutput()

                max1_ids = data.GetPointData().GetArray("Max1 ID")
                max2_ids = data.GetPointData().GetArray("Max2 ID")
                vals = data.GetPointData().GetArray("CP Value")
                sad_ids = data.GetPointData().GetArray("CP ID")
                max1_value = data.GetPointData().GetArray("Max1 Value")
                max2_value = data.GetPointData().GetArray("Max2 Value")

                cp3_ids = self.cp3_data.GetPointData().GetArray("CP ID")
                cp3_id_pos_dict = {}
                for i in range(self.cp3_data.GetNumberOfPoints()):
                    cp3_id_pos_dict[cp3_ids.GetValue(
                        i)] = self.cp3_data.GetPoint(i)

                numPoints = data.GetNumberOfPoints()
                list_triples = []
                all_nodes = set()
                # adding the position of saddle points
                for i in range(numPoints):
                    triple = Triples()
                    triple.cp_id1 = max1_ids.GetValue(i)
                    triple.pos1 = cp3_id_pos_dict[max1_ids.GetValue(i)]
                    triple.val1 = max1_value.GetValue(i)
                    all_nodes.add((triple.cp_id1, 3, triple.val1, 0))

                    triple.cp_id2 = sad_ids.GetValue(i)
                    triple.pos2 = data.GetPoint(i)
                    triple.val2 = vals.GetValue(i)
                    all_nodes.add((triple.cp_id2, 2, triple.val2, triple.alpha))

                    triple.cp_id3 = max2_ids.GetValue(i)
                    triple.pos3 = cp3_id_pos_dict[max2_ids.GetValue(i)]
                    triple.val3 = max2_value.GetValue(i)
                    all_nodes.add((triple.cp_id3, 3, triple.val3, 0))

                    if triple.val1 > triple.val3:
                        triple.alpha = 1 - (triple.val2 / triple.val3)
                    else:
                        triple.alpha = 1 - (triple.val2 / triple.val1)
                    list_triples.append(triple)

                self.nodes_new_id_dict = {}
                for i, node_data in enumerate(all_nodes):
                    self.nodes_new_id_dict[node_data[0]] = i

                node_feaures = self._get_node_features(all_nodes)
                edge_features = self._get_edge_features(list_triples)
                edge_index = self._get_adj_info(list_triples)
                g_label = self._get_label(int(label))

                data = Data(x=node_feaures,
                            edge_index=edge_index,
                            edge_attr=edge_features,
                            y=g_label,
                            dataset_name=folder_name,
                            file_name=file)

                torch.save(data, os.path.join(self.processed_dir,
                           f'data_{start_file_no + file_count}.pt'))
                self.files_metadata_dict[start_file_no + file_count] = {
                    'folder_name': folder_name, 'file_name': file, 'label': label}
                file_count += 1

        return file_count

    def _get_node_features(self, data):

        num_nodes = len(self.nodes_new_id_dict)
        all_nodes_features = [[] for i in range(num_nodes)]

        for i, node_data in enumerate(data):
            n_cp_id, n_type, n_value, n_apha = node_data
            n_new_id = self.nodes_new_id_dict[n_cp_id]
            all_nodes_features[n_new_id].append(n_type)
            all_nodes_features[n_new_id].append(n_value)
            all_nodes_features[n_new_id].append(n_apha)

        all_nodes_features = np.asarray(all_nodes_features)
        return torch.tensor(all_nodes_features, dtype=torch.float)

    def _get_edge_features(self, data):
        all_edge_features = []
        for i, triples in enumerate(data):
            edge1_a = [*triples.pos1]
            edge1_b = [*triples.pos2]
            edge2_a = [*triples.pos2]
            edge2_b = [*triples.pos3]
            all_edge_features.append(
                np.sqrt(np.sum(np.square(np.subtract(edge1_a, edge1_b)))))
            all_edge_features.append(
                np.sqrt(np.sum(np.square(np.subtract(edge2_a, edge2_b)))))

        all_edge_features = np.asarray(all_edge_features)
        return torch.tensor(all_edge_features, dtype=torch.float)

    def _get_adj_info(self, data):

        # 2 x num_edges
        all_edges = np.zeros((2, 2*len(data)))

        for i, triples in enumerate(data):
            n1_id = self.nodes_new_id_dict[triples.cp_id1]
            n2_id = self.nodes_new_id_dict[triples.cp_id2]
            n3_id = self.nodes_new_id_dict[triples.cp_id3]
            all_edges[0][2*i] = n1_id
            all_edges[1][2*i] = n2_id
            all_edges[0][2*i+1] = n2_id
            all_edges[1][2*i+1] = n3_id

        return torch.tensor(all_edges, dtype=torch.long)

    def _get_label(self, data):
        label = np.asarray([data])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt'))
        return data


# Training data

In [6]:
train_dataset = MorseGramDataset(root='data')

In [7]:
len(train_dataset)


1690

In [8]:
train_dataset[0]


Data(x=[84, 3], edge_index=[2, 96], edge_attr=[96], y=[1], dataset_name='odo', file_name='276574.vtp')

# Splitting Dataset

In [9]:
torch.manual_seed(12345)
dataset = train_dataset.shuffle()
num_data = len(dataset)
train_dataset = dataset[:int(num_data*0.8)]
test_dataset = dataset[int(num_data*0.8):]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 1352
Number of test graphs: 338


In [10]:
# different labels in test and train dataset
num_0 = 0
num_1 = 0
for i in range(len(train_dataset)):
    if train_dataset[i].y == 0:
        num_0 += 1
    else:
        num_1 += 1
print(f'Number of graphs with label 0 in train dataset: {num_0}')
print(f'Number of graphs with label 1 in train dataset: {num_1}')

num_0 = 0
num_1 = 0
for i in range(len(test_dataset)):
    if test_dataset[i].y == 0:
        num_0 += 1
    else:
        num_1 += 1
print(f'Number of graphs with label 0 in test dataset: {num_0}')
print(f'Number of graphs with label 1 in test dataset: {num_1}')

Number of graphs with label 0 in train dataset: 193
Number of graphs with label 1 in train dataset: 1159
Number of graphs with label 0 in test dataset: 60
Number of graphs with label 1 in test dataset: 278


# Dataloader

In [11]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[7080, 3], edge_index=[2, 8284], edge_attr=[8284], y=[64], dataset_name=[64], file_name=[64], batch=[7080], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[5447, 3], edge_index=[2, 6298], edge_attr=[6298], y=[64], dataset_name=[64], file_name=[64], batch=[5447], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[6189, 3], edge_index=[2, 7194], edge_attr=[7194], y=[64], dataset_name=[64], file_name=[64], batch=[6189], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[6209, 3], edge_index=[2, 7266], edge_attr=[7266], y=[64], dataset_name=[64], file_name=[64], batch=[6209], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[5968, 3], edge_index=[2, 6916], edge_attr=[6916], y=[64], dataset_name=[64], file_name=[64], batch=[5968], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[5718, 3], edge_index=[2, 6610], 

# Model

In [12]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        # x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = global_max_pool(x, batch)

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [13]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
logger1 = MLLogger('gcn')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    logger1.log_metrics(epoch=epoch,
                        test_accuracy=test_acc,
                        train_accuracy=train_acc,
                        test_loss=None,
                        train_loss=None)
    

# save logs
log_name = str(datetime.datetime.now()).replace(" ", "_").replace(":", "_").replace(".", "_")
logger1.save_log(log_name + ".csv")

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 002, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 003, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 004, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 005, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 006, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 007, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 008, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 009, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 010, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 011, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 012, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 013, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 014, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 015, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 016, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 017, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 018, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 019, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 020, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 021, Train Acc: 0.8572, Test Acc:

sample output

In [14]:
model_pred = model(test_dataset[0].x, test_dataset[0].edge_index, test_dataset[0].batch)


# GraphConv Model

In [15]:
from torch_geometric.nn import GraphConv


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # x = global_mean_pool(x, batch)
        x = global_max_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GNN(hidden_channels=64)
print(model)

GNN(
  (conv1): GraphConv(3, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [16]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GNN(hidden_channels=64)
logger2 = MLLogger('gnn')
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    logger2.log_metrics(epoch=epoch,
                        test_accuracy=test_acc,
                        train_accuracy=train_acc,
                        test_loss=None,
                        train_loss=None)
    
# save logs
log_name = str(datetime.datetime.now()).replace(" ", "_").replace(":", "_").replace(".", "_")
logger2.save_log(log_name + ".csv")

<IPython.core.display.Javascript object>

GNN(
  (conv1): GraphConv(3, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
Epoch: 001, Train Acc: 0.8565, Test Acc: 0.8225
Epoch: 002, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 003, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 004, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 005, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 006, Train Acc: 0.8565, Test Acc: 0.8225
Epoch: 007, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 008, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 009, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 010, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 011, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 012, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 013, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 014, Train Acc: 0.8565, Test Acc: 0.8225
Epoch: 015, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 016, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 017, Train Acc: 0.8572, Test Acc: 0.8225
Epoch: 018, Train Acc: 0.8565, T

# get particle which are wrongly classified

In [17]:
wrong_pred = []
for data in test_loader:
    out = model(data.x, data.edge_index, data.batch)
    pred = out.argmax(dim=1)
    for i in range(len(pred)):
        if pred[i] != data.y[i]:
            wrong_pred.append((data.y[i], pred[i], data.dataset_name[i], data.file_name[i]))
print(len(wrong_pred))
for wp in wrong_pred:
    print(wp)

58
(tensor(0), tensor(1), 'odo', '343503.vtp')
(tensor(0), tensor(1), 'odo', '358687.vtp')
(tensor(1), tensor(0), 'odo', '288510.vtp')
(tensor(0), tensor(1), 'odo', '208345.vtp')
(tensor(0), tensor(1), 'odo', '204808.vtp')
(tensor(1), tensor(0), 'odo', '527677.vtp')
(tensor(0), tensor(1), 'odo', '261809.vtp')
(tensor(0), tensor(1), 'odo', '546335.vtp')
(tensor(0), tensor(1), 'odo', '132346.vtp')
(tensor(1), tensor(0), 'odo', '151718.vtp')
(tensor(0), tensor(1), 'odo', '316046.vtp')
(tensor(0), tensor(1), 'odo', '178677.vtp')
(tensor(0), tensor(1), 'odo', '522249.vtp')
(tensor(0), tensor(1), 'odo', '118026.vtp')
(tensor(0), tensor(1), 'odo', '532689.vtp')
(tensor(1), tensor(0), 'odo', '467331.vtp')
(tensor(0), tensor(1), 'odo', '339686.vtp')
(tensor(0), tensor(1), 'odo', '213507.vtp')
(tensor(0), tensor(1), 'odo', '257888.vtp')
(tensor(0), tensor(1), 'odo', '447626.vtp')
(tensor(0), tensor(1), 'odo', '557818.vtp')
(tensor(0), tensor(1), 'odo', '537408.vtp')
(tensor(0), tensor(1), 'odo',

# visualize the model